In [2]:
import os
import numpy as np
import pandas as pd

import datetime as dt
from datetime import datetime, timedelta, time
from pandas import DataFrame

In [2]:
#데이터 불러오기
path = "/root/yerim/"
os.chdir(path)


train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')
log_train = pd.read_csv('data/log_train.csv')
log_test = pd.read_csv('data/log_test.csv')

In [3]:
print(log_train.shape)
print(log_test.shape)
print(train.shape)
print(test.shape)

(12907328, 6)
(4936665, 6)
(10264349, 25)
(3255482, 24)


In [4]:
#날짜형 변환
train['insert_time']=pd.to_datetime(train['insert_time'],format="%Y-%m-%d %H:%M:%S")
log_train['timestamp']=pd.to_datetime(log_train['timestamp'],format="%Y-%m-%d %H:%M:%S")
log_train['date_cd']=pd.to_datetime(log_train['date_cd'],format="%Y-%m-%d")

## train 데이터 처리

In [5]:
train.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,isjob,age
0,566528,2022-03-01 00:11:36,13,123,20000000.0,19.1,0.0,681184,1974.0,1.0,...,기타,전월세,1000000.0,LIVING,before_update,before_update,4.0,20000000.0,0,49.0
1,38545,2022-03-01 00:14:15,55,12,27000000.0,16.4,0.0,477093,1971.0,0.0,...,기타,기타가족소유,4000000.0,LIVING,before_update,before_update,6.0,30000000.0,0,52.0
2,182468,2022-03-01 00:15:52,35,29,3000000.0,17.9,0.0,92495,1980.0,0.0,...,기타,기타가족소유,30000000.0,SWITCHLOAN,before_update,before_update,3.0,27000000.0,0,43.0
3,182468,2022-03-01 00:15:52,55,12,20000000.0,13.0,0.0,92495,1980.0,0.0,...,기타,기타가족소유,30000000.0,SWITCHLOAN,before_update,before_update,3.0,27000000.0,0,43.0
4,182468,2022-03-01 00:15:52,14,128,6000000.0,14.9,0.0,92495,1980.0,0.0,...,기타,기타가족소유,30000000.0,SWITCHLOAN,before_update,before_update,3.0,27000000.0,0,43.0


In [6]:
#사용할 컬럼만 선택. 하나의 application_id는 모두 같은 insert_time을 가짐
train_new = train[['application_id','user_id','insert_time']]

In [7]:
#하나의 application_id가 여러 개의 product_id에 대응되므로 unique한 application_id만 뽑아냄
train_new = train_new.drop_duplicates(['application_id','user_id','insert_time'],keep="first")

In [8]:
train_new.shape #train 데이터에 있는 application_id의 수는 총 728,860개

(728860, 3)

In [9]:
train['application_id'].nunique()

728860

In [10]:
train_new['event'] = 'APPLY' #train에 기록된 각 applicaton_id의 insert_time을 'APPLY'한 시각이라고 정의

In [11]:
train_new.rename(columns = {'insert_time':'timestamp'},inplace=True) #이후 merge를 하기 위해 컬럼명 바꾸기

In [12]:
train_new

,application_id,user_id,timestamp,event
0,566528,681184,2022-03-01 00:11:35,APPLY
1,38545,477093,2022-03-01 00:14:14,APPLY
2,182468,92495,2022-03-01 00:15:51,APPLY
12,1641514,477093,2022-03-01 00:16:46,APPLY
13,1799957,532198,2022-03-01 00:18:35,APPLY
...,...,...,...,...
10264325,200527,76803,2022-05-27 14:19:58,APPLY
10264335,507566,718601,2022-05-27 19:40:39,APPLY
10264336,83162,313726,2022-05-29 09:08:27,APPLY
10264344,247438,199299,2022-05-30 16:40:56,APPLY


## log 데이터 처리

In [13]:
log_train

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
12907323,242442,StartLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
12907324,242442,EndLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
12907325,593062,OpenApp,2022-05-27 16:19:10,Android,3.13.0,2022-05-27
12907326,593062,Login,2022-05-27 16:19:15,Android,3.13.0,2022-05-27


In [14]:
log_train['mp_os'].value_counts()

Android    8975435
iOS        2456204
android    1473228
iPadOS        1677
Name: mp_os, dtype: int64

In [15]:
log_train[log_train['mp_os'].isna()]['event']

417634      GetCreditInfo
422048      GetCreditInfo
422050      GetCreditInfo
422124      GetCreditInfo
424967      GetCreditInfo
                ...      
12434999    GetCreditInfo
12436654    GetCreditInfo
12436757    GetCreditInfo
12436768    GetCreditInfo
12436842    GetCreditInfo
Name: event, Length: 784, dtype: object

mp_os가 결측이면, event가 모두 GetCreditInfo =>오류라고 판단

In [16]:
log_train.loc[log_train['mp_os'].isna(), 'event'] = 'Error'

mp_os가 NA인 경우 활동로그가 제대로 입력되지 않는 것 같으므로 Error로 표기

In [17]:
log_train['mp_os'].isna().sum()

784

In [18]:
log_train.loc[log_train['mp_os'].isna(), 'mp_os'] = 'unknown' #로그데이터에 아예 한번도 기록이 없던 유저와 구분하기 위함

In [19]:
log_train[log_train['mp_os']=='android']['event'].nunique()

1

mp_os가 android이면 event가 모두 GetCreditInfo => 오류라고 판단

In [20]:
log_train.loc[log_train['mp_os']=='android', 'event'] = 'Error2'

In [21]:
log_train.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25


In [22]:
log_train = log_train.sort_values(['user_id','timestamp'])

In [23]:
log_train

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
8985228,1,Error2,2022-05-03 14:52:28,android,464,2022-05-03
8985230,1,Error2,2022-05-03 14:52:35,android,464,2022-05-03
8133626,7,Error2,2022-05-22 16:39:49,android,465,2022-05-22
7532851,9,Error2,2022-05-21 23:37:58,android,465,2022-05-21
7532880,9,Error2,2022-05-21 23:43:33,android,465,2022-05-21
...,...,...,...,...,...,...
5772878,879696,CompleteIDCertification,2022-03-14 05:38:16,Android,3.8.2,2022-03-14
5772879,879696,EndLoanApply,2022-03-14 05:42:14,Android,3.8.2,2022-03-14
6553999,879698,OpenApp,2022-05-24 22:33:24,iOS,NaN,2022-05-24
6554002,879698,StartLoanApply,2022-05-24 22:33:32,iOS,NaN,2022-05-24


## Session 나누기(30분동안 아무런 행동이 없으면 Session 종료라고 판단)

In [24]:
log_train.groupby(['user_id','date_cd'])['timestamp'].diff(axis=0)

8985228               NaT
8985230   0 days 00:00:07
8133626               NaT
7532851               NaT
7532880   0 days 00:05:35
                ...      
5772878   0 days 00:00:21
5772879   0 days 00:03:58
6553999               NaT
6554002   0 days 00:00:08
6554004   0 days 00:00:00
Name: timestamp, Length: 12907328, dtype: timedelta64[ns]

In [25]:
log_train['timediff'] = log_train.groupby(['user_id','date_cd'])['timestamp'].diff(axis=0)
log_train['timediff_min'] = log_train['timediff'].dt.total_seconds()/60 #분으로 표현

In [26]:
def func(x) :
    if x > 30 :
        return "Session_End"
    else :
        return np.nan
    
log_train["Session"] = log_train["timediff_min"].apply(lambda x : func(x))

In [27]:
log_train[log_train['Session']=='Session_End']

,user_id,event,timestamp,mp_os,mp_app_version,date_cd,timediff,timediff_min,Session
4556297,12,OpenApp,2022-05-01 18:30:33,Android,3.11.1,2022-05-01,0 days 00:39:34,39.566667,Session_End
5706140,17,OpenApp,2022-03-07 13:29:23,iOS,3.6.1,2022-03-07,0 days 06:08:13,368.216667,Session_End
3702232,20,OpenApp,2022-03-15 17:18:00,Android,3.8.2,2022-03-15,0 days 00:56:28,56.466667,Session_End
8080623,20,OpenApp,2022-03-17 16:18:03,Android,3.8.2,2022-03-17,0 days 12:22:44,742.733333,Session_End
8080676,20,OpenApp,2022-03-17 17:14:47,Android,3.8.2,2022-03-17,0 days 00:51:51,51.850000,Session_End
...,...,...,...,...,...,...,...,...,...
3291910,879691,OpenApp,2022-03-13 14:35:34,Android,3.8.2,2022-03-13,0 days 01:13:13,73.216667,Session_End
3453470,879692,OpenApp,2022-04-20 17:26:50,Android,3.10.1,2022-04-20,0 days 04:32:07,272.116667,Session_End
12362714,879692,EndLoanApply,2022-04-22 17:12:47,Android,3.10.1,2022-04-22,0 days 06:24:38,384.633333,Session_End
1654590,879693,OpenApp,2022-05-13 13:24:23,iOS,3.10.2,2022-05-13,0 days 01:43:42,103.700000,Session_End


In [28]:
log_train['Session_point'] = np.where(log_train['Session']=='Session_End', 1,0)

In [29]:
log_train['Session_cum'] = log_train.groupby(['user_id','date_cd'])['Session_point'].cumsum()

In [30]:
log_train['Session_cum'] = log_train['Session_cum']+1

In [31]:
log_train

,user_id,event,timestamp,mp_os,mp_app_version,date_cd,timediff,timediff_min,Session,Session_point,Session_cum
8985228,1,Error2,2022-05-03 14:52:28,android,464,2022-05-03,NaT,NaN,NaN,0,1
8985230,1,Error2,2022-05-03 14:52:35,android,464,2022-05-03,0 days 00:00:07,0.116667,NaN,0,1
8133626,7,Error2,2022-05-22 16:39:49,android,465,2022-05-22,NaT,NaN,NaN,0,1
7532851,9,Error2,2022-05-21 23:37:58,android,465,2022-05-21,NaT,NaN,NaN,0,1
7532880,9,Error2,2022-05-21 23:43:33,android,465,2022-05-21,0 days 00:05:35,5.583333,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...
5772878,879696,CompleteIDCertification,2022-03-14 05:38:16,Android,3.8.2,2022-03-14,0 days 00:00:21,0.350000,NaN,0,1
5772879,879696,EndLoanApply,2022-03-14 05:42:14,Android,3.8.2,2022-03-14,0 days 00:03:58,3.966667,NaN,0,1
6553999,879698,OpenApp,2022-05-24 22:33:24,iOS,NaN,2022-05-24,NaT,NaN,NaN,0,1
6554002,879698,StartLoanApply,2022-05-24 22:33:32,iOS,NaN,2022-05-24,0 days 00:00:08,0.133333,NaN,0,1


In [32]:
log_train = log_train.drop(['timediff','timediff_min','Session','Session_point'],axis=1)

In [33]:
log_train.rename(columns={'Session_cum':'Session'},inplace=True)

In [34]:
log_train

,user_id,event,timestamp,mp_os,mp_app_version,date_cd,Session
8985228,1,Error2,2022-05-03 14:52:28,android,464,2022-05-03,1
8985230,1,Error2,2022-05-03 14:52:35,android,464,2022-05-03,1
8133626,7,Error2,2022-05-22 16:39:49,android,465,2022-05-22,1
7532851,9,Error2,2022-05-21 23:37:58,android,465,2022-05-21,1
7532880,9,Error2,2022-05-21 23:43:33,android,465,2022-05-21,1
...,...,...,...,...,...,...,...
5772878,879696,CompleteIDCertification,2022-03-14 05:38:16,Android,3.8.2,2022-03-14,1
5772879,879696,EndLoanApply,2022-03-14 05:42:14,Android,3.8.2,2022-03-14,1
6553999,879698,OpenApp,2022-05-24 22:33:24,iOS,NaN,2022-05-24,1
6554002,879698,StartLoanApply,2022-05-24 22:33:32,iOS,NaN,2022-05-24,1


In [35]:
log_train = log_train[['user_id','event','timestamp','date_cd','mp_os','Session']]

In [36]:
log_train = log_train.sort_values(['user_id','timestamp'])

In [37]:
log_train['application_id']=np.nan #merge하기 위해 application_id컬럼 만들어주기

In [38]:
log_train = log_train[['application_id','user_id','timestamp','date_cd','event','mp_os','Session']]

In [39]:
log_train.head(3)

,application_id,user_id,timestamp,date_cd,event,mp_os,Session
8985228,NaN,1,2022-05-03 14:52:28,2022-05-03,Error2,android,1
8985230,NaN,1,2022-05-03 14:52:35,2022-05-03,Error2,android,1
8133626,NaN,7,2022-05-22 16:39:49,2022-05-22,Error2,android,1


## train과 log 데이터 concat

In [40]:
log_train

,application_id,user_id,timestamp,date_cd,event,mp_os,Session
8985228,NaN,1,2022-05-03 14:52:28,2022-05-03,Error2,android,1
8985230,NaN,1,2022-05-03 14:52:35,2022-05-03,Error2,android,1
8133626,NaN,7,2022-05-22 16:39:49,2022-05-22,Error2,android,1
7532851,NaN,9,2022-05-21 23:37:58,2022-05-21,Error2,android,1
7532880,NaN,9,2022-05-21 23:43:33,2022-05-21,Error2,android,1
...,...,...,...,...,...,...,...
5772878,NaN,879696,2022-03-14 05:38:16,2022-03-14,CompleteIDCertification,Android,1
5772879,NaN,879696,2022-03-14 05:42:14,2022-03-14,EndLoanApply,Android,1
6553999,NaN,879698,2022-05-24 22:33:24,2022-05-24,OpenApp,iOS,1
6554002,NaN,879698,2022-05-24 22:33:32,2022-05-24,StartLoanApply,iOS,1


In [41]:
log_train.drop(['date_cd'],axis=1,inplace=True)

In [42]:
train_new

,application_id,user_id,timestamp,event
0,566528,681184,2022-03-01 00:11:35,APPLY
1,38545,477093,2022-03-01 00:14:14,APPLY
2,182468,92495,2022-03-01 00:15:51,APPLY
12,1641514,477093,2022-03-01 00:16:46,APPLY
13,1799957,532198,2022-03-01 00:18:35,APPLY
...,...,...,...,...
10264325,200527,76803,2022-05-27 14:19:58,APPLY
10264335,507566,718601,2022-05-27 19:40:39,APPLY
10264336,83162,313726,2022-05-29 09:08:27,APPLY
10264344,247438,199299,2022-05-30 16:40:56,APPLY


In [43]:
train_new['mp_os']=np.nan #데이터 합치기 위해 mp_os칼럼 만들어주기
train_new['Session']=np.nan

In [44]:
train_new.head(3)

,application_id,user_id,timestamp,event,mp_os,Session
0,566528,681184,2022-03-01 00:11:35,APPLY,NaN,NaN
1,38545,477093,2022-03-01 00:14:14,APPLY,NaN,NaN
2,182468,92495,2022-03-01 00:15:51,APPLY,NaN,NaN


In [45]:
#tlog데이터에 해당 application이 APPLY'된 시점을 넣기 위해 concat함
concat_train = pd.concat([log_train,train_new],ignore_index=True)

In [46]:
concat_train

,application_id,user_id,timestamp,event,mp_os,Session
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0
3,NaN,9,2022-05-21 23:37:58,Error2,android,1.0
4,NaN,9,2022-05-21 23:43:33,Error2,android,1.0
...,...,...,...,...,...,...
13636183,200527.0,76803,2022-05-27 14:19:58,APPLY,NaN,NaN
13636184,507566.0,718601,2022-05-27 19:40:39,APPLY,NaN,NaN
13636185,83162.0,313726,2022-05-29 09:08:27,APPLY,NaN,NaN
13636186,247438.0,199299,2022-05-30 16:40:56,APPLY,NaN,NaN


In [47]:
concat_train = concat_train.sort_values(['user_id','timestamp'])

In [48]:
concat_train

,application_id,user_id,timestamp,event,mp_os,Session
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0
3,NaN,9,2022-05-21 23:37:58,Error2,android,1.0
13563085,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,NaN
...,...,...,...,...,...,...
12907324,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0
13474911,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,NaN
12907325,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0
12907326,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0


In [49]:
#timestamp에서 시간을 제외하고 날짜만 따로 뽑아서 date컬럼을 만듦
concat_train['date'] = concat_train['timestamp'].dt.date

In [50]:
concat_train.shape

(13636188, 7)

In [51]:
concat_train.isna().sum()

application_id    12907328
user_id                  0
timestamp                0
event                    0
mp_os               728860
Session             728860
date                     0
dtype: int64

In [52]:
# train데이터였던 행의 값을 채워주기 위해 해당 행의 위 값으로 채워줌 (바로 위 행동과 30분 이상 차이나면 다른 세션이므로 채우지 않음)
#concat_train['Session'] = concat_train.groupby(['user_id','date'])['Session'].ffill()
concat_train['Session'] = np.where(concat_train.groupby(['user_id','date'])['timestamp'].diff(axis=0).dt.total_seconds()/60 <30, concat_train['Session'].ffill(), concat_train['Session'])

In [53]:
concat_train[concat_train['Session'].isna()]

,application_id,user_id,timestamp,event,mp_os,Session,date
13313697,1508745.0,14,2022-04-18 14:37:09,APPLY,NaN,NaN,2022-04-18
12956556,1854180.0,19,2022-05-06 17:10:55,APPLY,NaN,NaN,2022-05-06
13056033,513514.0,20,2022-03-13 20:17:19,APPLY,NaN,NaN,2022-03-13
12918075,741752.0,48,2022-03-15 21:18:51,APPLY,NaN,NaN,2022-03-15
13395092,390821.0,51,2022-04-29 07:01:11,APPLY,NaN,NaN,2022-04-29
...,...,...,...,...,...,...,...
13160464,986932.0,879659,2022-03-28 15:11:37,APPLY,NaN,NaN,2022-03-28
13307482,1607051.0,879685,2022-04-17 21:25:58,APPLY,NaN,NaN,2022-04-17
13291757,1057391.0,879692,2022-04-14 20:13:12,APPLY,NaN,NaN,2022-04-14
13057210,2149910.0,879696,2022-03-14 05:23:30,APPLY,NaN,NaN,2022-03-14


In [54]:
concat_train.isna().sum()

application_id    12907328
user_id                  0
timestamp                0
event                    0
mp_os               728860
Session              83399
date                     0
dtype: int64

In [55]:
concat_train['Session'].fillna(0,inplace=True) #해당 활동로그가 없어서 Session이 기록되지 않은 경우로 이 경우는 0으로 채움

In [56]:
concat_train

,application_id,user_id,timestamp,event,mp_os,Session,date
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22
3,NaN,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21
13563085,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,1.0,2022-05-21
...,...,...,...,...,...,...,...
12907324,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14
13474911,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,0.0,2022-05-10
12907325,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24
12907326,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24


## application_id 채우기

In [57]:
concat_train2 = concat_train.copy()

In [58]:
concat_train2 = concat_train2.sort_values(['user_id','timestamp'])

In [59]:
concat_train2['application_id'] = concat_train2.groupby(['user_id','date','Session'])['application_id'].bfill()

In [60]:
concat_train2['application_id'].nunique()

728860

In [228]:
concat_train2.head(30)

,application_id,user_id,timestamp,event,mp_os,Session,date,first_visit,how_long
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,2022-05-03,1
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,2022-05-03,1
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,2022-05-22,1
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,2022-05-21,1
13563085,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,1.0,2022-05-21,2022-05-21,1
4,NaN,9,2022-05-21 23:43:33,Error2,android,1.0,2022-05-21,2022-05-21,1
5,NaN,9,2022-05-21 23:43:52,Error2,android,1.0,2022-05-21,2022-05-21,1
6,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,2022-03-24,1
7,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,2022-03-24,1
8,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,2022-03-24,1


## 처음 방문으로부터 며칠이 지났는가? (how_long) (1부터 시작)

In [61]:
concat_train2['first_visit'] = concat_train2.sort_values(by=['user_id', 'date']).groupby('user_id').date.transform('first')

In [62]:
concat_train2['how_long']=concat_train2['date']-concat_train2['first_visit']

In [63]:
concat_train2.sort_values('how_long',ascending=False)

,application_id,user_id,timestamp,event,mp_os,Session,date,first_visit,how_long
11039343,543462.0,752343,2022-05-31 16:56:10,StartLoanApply,Android,1.0,2022-05-31,2022-03-01,91 days
5878748,524218.0,401889,2022-05-31 12:57:07,UseLoanManage,Android,1.0,2022-05-31,2022-03-01,91 days
161771,NaN,11532,2022-05-31 22:26:51,Login,Android,1.0,2022-05-31,2022-03-01,91 days
161772,NaN,11532,2022-05-31 22:26:54,UseLoanManage,Android,1.0,2022-05-31,2022-03-01,91 days
5390337,2073806.0,369094,2022-05-31 09:34:49,OpenApp,Android,1.0,2022-05-31,2022-03-01,91 days
...,...,...,...,...,...,...,...,...,...
7124954,NaN,485861,2022-03-03 18:14:25,OpenApp,Android,1.0,2022-03-03,2022-03-03,0 days
7124955,NaN,485861,2022-03-03 18:14:30,Login,Android,1.0,2022-03-03,2022-03-03,0 days
7124956,NaN,485861,2022-03-03 18:14:40,UseLoanManage,Android,1.0,2022-03-03,2022-03-03,0 days
7124957,NaN,485861,2022-03-03 18:14:42,Error2,android,1.0,2022-03-03,2022-03-03,0 days


In [64]:
concat_train2['how_long'] = concat_train2['how_long'].dt.days
concat_train2['how_long'] = concat_train2['how_long']+1

In [65]:
concat_train2

,application_id,user_id,timestamp,event,mp_os,Session,date,first_visit,how_long
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,2022-05-03,1
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,2022-05-03,1
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,2022-05-22,1
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,2022-05-21,1
13563085,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,1.0,2022-05-21,2022-05-21,1
...,...,...,...,...,...,...,...,...,...
12907324,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14,2022-03-14,1
13474911,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,0.0,2022-05-10,2022-05-10,1
12907325,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24,2022-05-24,1
12907326,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24,2022-05-24,1


## 누적 방문횟수가 얼마인가? (visit_cumsum)

In [66]:
concat_train3 = concat_train2.copy()

In [67]:
concat_train3 = concat_train3.sort_values(['user_id','timestamp'])

In [68]:
concat_train3 = concat_train3[['user_id','date']]

In [69]:
concat_train3['visit']=1

In [70]:
concat_train3 = concat_train3.drop_duplicates(['user_id','date','visit'],keep="first")

In [71]:
concat_train3

,user_id,date,visit
0,1,2022-05-03,1
2,7,2022-05-22,1
3,9,2022-05-21,1
6,11,2022-03-24,1
18,11,2022-04-20,1
...,...,...,...
12907304,879694,2022-03-31,1
12907310,879695,2022-05-27,1
13057210,879696,2022-03-14,1
13474911,879697,2022-05-10,1


In [72]:
concat_train3['visit_sum'] = concat_train3.groupby('user_id')['visit'].cumsum()

In [73]:
concat_train3['visit_cumsum'] = concat_train3.groupby('user_id')['visit_sum'].shift(1) #전날까지 방문횟수이므로 shift(1)

In [74]:
concat_train3['visit_cumsum'].fillna(0,inplace=True)

In [75]:
concat_train3 = concat_train3.drop(['visit_sum'],axis=1)

In [76]:
concat_train3

,user_id,date,visit,visit_cumsum
0,1,2022-05-03,1,0.0
2,7,2022-05-22,1,0.0
3,9,2022-05-21,1,0.0
6,11,2022-03-24,1,0.0
18,11,2022-04-20,1,1.0
...,...,...,...,...
12907304,879694,2022-03-31,1,0.0
12907310,879695,2022-05-27,1,0.0
13057210,879696,2022-03-14,1,0.0
13474911,879697,2022-05-10,1,0.0


## 지난 방문으로부터 며칠만의 방문인가? (from_last_visit)

In [77]:
concat_train3['from_last_visit']=concat_train3.groupby('user_id')['date'].diff(axis=0)

In [78]:
concat_train3['from_last_visit'] = concat_train3['from_last_visit'].dt.days

In [79]:
concat_train3

,user_id,date,visit,visit_cumsum,from_last_visit
0,1,2022-05-03,1,0.0,NaN
2,7,2022-05-22,1,0.0,NaN
3,9,2022-05-21,1,0.0,NaN
6,11,2022-03-24,1,0.0,NaN
18,11,2022-04-20,1,1.0,27.0
...,...,...,...,...,...
12907304,879694,2022-03-31,1,0.0,NaN
12907310,879695,2022-05-27,1,0.0,NaN
13057210,879696,2022-03-14,1,0.0,NaN
13474911,879697,2022-05-10,1,0.0,NaN


In [80]:
concat_train3['from_last_visit'].fillna(0,inplace=True) #이전 방문이 없어 NA인 경우, 0으로 대체

## 해당 일 기준, 지난 7일동안 한 번이라도 방문했는가? (했으면 1 안 했으면 0)

In [81]:
concat_train3['before_7days'] = np.where((concat_train3['from_last_visit']<=7)&(concat_train3['from_last_visit']!=0), 1, 0)

In [82]:
concat_train3[concat_train3['before_7days']==1]

,user_id,date,visit,visit_cumsum,from_last_visit,before_7days
70,12,2022-05-01,1,4.0,6.0,1
84,12,2022-05-05,1,5.0,4.0,1
93,12,2022-05-10,1,6.0,5.0,1
102,12,2022-05-11,1,7.0,1.0,1
110,12,2022-05-16,1,8.0,5.0,1
...,...,...,...,...,...,...
12907226,879692,2022-04-20,1,2.0,2.0,1
12907238,879692,2022-04-22,1,3.0,2.0,1
12907252,879692,2022-04-25,1,4.0,3.0,1
12907276,879693,2022-05-17,1,1.0,4.0,1


## 해당 일 기준, 지난 30일동안 한 번이라도 방문했는가? (했으면 1, 안 했으면 0)

In [83]:
concat_train3['before_30days'] = np.where((concat_train3['from_last_visit']<=30)&(concat_train3['from_last_visit']!=0), 1, 0)

In [84]:
concat_train4 = pd.merge(concat_train2, concat_train3, how='left', on=['user_id','date'])

In [85]:
concat_train4

,application_id,user_id,timestamp,event,mp_os,Session,date,first_visit,how_long,visit,visit_cumsum,from_last_visit,before_7days,before_30days
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,2022-05-03,1,1,0.0,0.0,0,0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,2022-05-03,1,1,0.0,0.0,0,0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,2022-05-22,1,1,0.0,0.0,0,0
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,2022-05-21,1,1,0.0,0.0,0,0
4,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,1.0,2022-05-21,2022-05-21,1,1,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636183,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14,2022-03-14,1,1,0.0,0.0,0,0
13636184,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,0.0,2022-05-10,2022-05-10,1,1,0.0,0.0,0,0
13636185,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24,2022-05-24,1,1,0.0,0.0,0,0
13636186,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24,2022-05-24,1,1,0.0,0.0,0,0


## 해당 날짜 이전까지 총 방문율은? (어제까지 방문한 일 수 / 처음 방문한 이후부터 오늘까지 일 수)

In [86]:
concat_train4['visit_rate'] = concat_train4['visit_cumsum'] / concat_train4['how_long']

In [87]:
concat_train4.drop(['first_visit'],axis=1,inplace=True)

In [88]:
concat_train4[concat_train4['visit_rate']>0]

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate
20,1751522.0,11,2022-04-20 10:04:10,OpenApp,iOS,1.0,2022-04-20,28,1,1.0,27.0,0,1,0.035714
21,1751522.0,11,2022-04-20 10:04:16,GetCreditInfo,iOS,1.0,2022-04-20,28,1,1.0,27.0,0,1,0.035714
22,1751522.0,11,2022-04-20 10:04:17,UseLoanManage,iOS,1.0,2022-04-20,28,1,1.0,27.0,0,1,0.035714
23,1751522.0,11,2022-04-20 10:09:23,ViewLoanApplyIntro,iOS,1.0,2022-04-20,28,1,1.0,27.0,0,1,0.035714
24,1751522.0,11,2022-04-20 10:09:37,CompleteIDCertification,iOS,1.0,2022-04-20,28,1,1.0,27.0,0,1,0.035714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636155,NaN,879693,2022-05-20 16:36:40,StartLoanApply,iOS,1.0,2022-05-20,8,1,2.0,3.0,1,1,0.250000
13636156,NaN,879693,2022-05-20 16:36:41,EndLoanApply,iOS,1.0,2022-05-20,8,1,2.0,3.0,1,1,0.250000
13636157,NaN,879693,2022-05-20 16:36:43,UseLoanManage,iOS,1.0,2022-05-20,8,1,2.0,3.0,1,1,0.250000
13636158,NaN,879693,2022-05-20 16:39:53,StartLoanApply,iOS,1.0,2022-05-20,8,1,2.0,3.0,1,1,0.250000


In [89]:
concat_train4.isna().sum()

application_id     9302325
user_id                  0
timestamp                0
event                    0
mp_os               728860
Session                  0
date                     0
how_long                 0
visit                    0
visit_cumsum             0
from_last_visit          0
before_7days             0
before_30days            0
visit_rate               0
dtype: int64

In [90]:
concat_train4.head()

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,1,1,0.0,0.0,0,0,0.0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,1,1,0.0,0.0,0,0,0.0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,1,1,0.0,0.0,0,0,0.0
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,1,0.0,0.0,0,0,0.0
4,248576.0,9,2022-05-21 23:41:09,APPLY,NaN,1.0,2022-05-21,1,1,0.0,0.0,0,0,0.0


## mp_os 채우기

In [91]:
concat_train4 = concat_train4.sort_values(['user_id','timestamp'])

In [92]:
#mp_os를 같은 user, 같은 date 내에서 앞에 있는 값으로 채우기
concat_train4['mp_os']=concat_train4.groupby(['user_id','date'])['mp_os'].ffill()

In [93]:
concat_train4[concat_train4['mp_os'].isna()]['application_id'].nunique()

83114

In [94]:
concat_train4['mp_os'] = concat_train4.groupby('user_id')['mp_os'].ffill()

In [95]:
concat_train4[concat_train4['mp_os'].isna()]['application_id'].nunique()

70103

In [96]:
concat_train4['mp_os'] = concat_train4.groupby('user_id')['mp_os'].bfill()

In [97]:
concat_train4[concat_train4['mp_os'].isna()]['application_id'].nunique()

24273

In [98]:
concat_train4[concat_train4['mp_os'].isna()]['user_id'].nunique()

22280

In [99]:
concat_train4.drop(['visit'],axis=1,inplace=True)

## 지금까지 몇 번의 세션을 총 실행했는가? (session_sum)

In [100]:
concat_train5 = concat_train4.copy()

In [101]:
concat_train5['last_session'] = concat_train5.sort_values(by=['user_id', 'timestamp']).groupby(['user_id','date']).Session.transform('last')

In [102]:
concat_train5

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,last_session
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.0,1.0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.0,1.0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,1,0.0,0.0,0,0,0.0,1.0
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,1.0
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636183,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,1.0
13636184,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,0.0,2022-05-10,1,0.0,0.0,0,0,0.0,0.0
13636185,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24,1,0.0,0.0,0,0,0.0,1.0
13636186,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24,1,0.0,0.0,0,0,0.0,1.0


In [103]:
concat_train5 = concat_train5[['user_id','date','last_session']]

In [104]:
concat_train5 = concat_train5.drop_duplicates(['user_id','date','last_session'],keep="first")

In [105]:
concat_train5['last_session'] = np.where(concat_train5['last_session']==0, 1,concat_train5['last_session'])

In [106]:
concat_train5['session_cum']=concat_train5.groupby('user_id')['last_session'].cumsum()

In [107]:
concat_train5['session_sum'] = concat_train5.groupby('user_id')['session_cum'].shift(1) #이전의 총 session 수 이므로

In [108]:
concat_train5['session_sum'].fillna(0,inplace=True)

In [109]:
concat_train5 = concat_train5.drop(['last_session','session_cum'],axis=1)

In [110]:
concat_train5

,user_id,date,session_sum
0,1,2022-05-03,0.0
2,7,2022-05-22,0.0
3,9,2022-05-21,0.0
7,11,2022-03-24,0.0
20,11,2022-04-20,1.0
...,...,...,...
13636160,879694,2022-03-31,0.0
13636166,879695,2022-05-27,0.0
13636171,879696,2022-03-14,0.0
13636184,879697,2022-05-10,0.0


In [111]:
concat_train6 = pd.merge(concat_train4, concat_train5, how='left', on=['user_id','date'])

In [112]:
concat_train6

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.0,0.0
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.0,0.0
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,1,0.0,0.0,0,0,0.0,0.0
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636183,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0
13636184,1543548.0,879697,2022-05-10 20:40:52,APPLY,NaN,0.0,2022-05-10,1,0.0,0.0,0,0,0.0,0.0
13636185,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24,1,0.0,0.0,0,0,0.0,0.0
13636186,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24,1,0.0,0.0,0,0,0.0,0.0


## 평균적으로 몇 번의 session을 사용하는가? (session_avg)
### (어제까지 실행한 총 session의 수 / 어제까지의 총 방문 수)

In [113]:
concat_train6['session_avg']=concat_train6['session_sum']/concat_train6['visit_cumsum']

In [114]:
concat_train6['session_avg'].fillna(0,inplace=True)

## 신청서를 작성하기 전 마지막으로 방문한 시간~신청서 작성까지 얼마나? (from_last_visit_time)

In [115]:
concat_train7 = concat_train6.copy()

In [116]:
concat_train7['from_last_visit_time'] = concat_train7.groupby('user_id')['timestamp'].diff(axis=0)

In [117]:
concat_train7.head(30)

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,from_last_visit_time
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.000000,0.0,0.0,NaT
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:00:07
2,NaN,7,2022-05-22 16:39:49,Error2,android,1.0,2022-05-22,1,0.0,0.0,0,0,0.000000,0.0,0.0,NaT
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,NaT
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:03:11
5,NaN,9,2022-05-21 23:43:33,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:02:24
6,NaN,9,2022-05-21 23:43:52,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:00:19
7,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,NaT
8,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:00:08
9,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,0 days 00:00:01


In [118]:
concat_train7['from_last_visit_time2'] = concat_train7.groupby(['user_id','application_id']).from_last_visit_time.head(1)

In [119]:
concat_train7 = concat_train7[concat_train7['application_id'].notna()]

In [120]:
concat_train7 = concat_train7[['application_id','user_id','from_last_visit_time2']]

In [121]:
concat_train7= concat_train7.drop_duplicates(['application_id','user_id'],keep="first")

In [122]:
concat_train7.rename(columns={'from_last_visit_time2':'from_last_visit_time'},inplace=True)

In [123]:
concat_train7

,application_id,user_id,from_last_visit_time
3,248576.0,9,NaT
7,1180616.0,11,NaT
20,1751522.0,11,26 days 22:58:39
128,1508745.0,14,NaT
129,1990215.0,17,NaT
...,...,...,...
13636147,716861.0,879693,3 days 02:09:34
13636166,1774704.0,879695,NaT
13636171,2149910.0,879696,NaT
13636172,1856758.0,879696,0 days 00:12:04


## train데이터에 있는 application_id에 해당하는 데이터만 추출

In [124]:
#train데이터는 모두 application_id가 있으므로 application_id가 결측이 아닌 행들만 가져옴
concat_train_fin = concat_train6[concat_train6['application_id'].notna()]

In [125]:
concat_train_fin['application_id'].nunique()

728860

In [126]:
concat_train_fin

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0
7,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
8,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
9,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636179,1856758.0,879696,2022-03-14 05:37:39,Login,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636180,1856758.0,879696,2022-03-14 05:37:55,ViewLoanApplyIntro,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636181,1856758.0,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636182,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0


In [127]:
concat_train_fin = concat_train_fin.drop_duplicates(['user_id','timestamp','event'],keep="first")

In [128]:
concat_train_fin

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0
7,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
8,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
9,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636179,1856758.0,879696,2022-03-14 05:37:39,Login,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636180,1856758.0,879696,2022-03-14 05:37:55,ViewLoanApplyIntro,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636181,1856758.0,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0
13636182,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0


#### event 행동 별 횟수 변수 만들기(1)

In [129]:
train_dummy = pd.get_dummies(concat_train_fin['event'], dummy_na=False)

In [130]:
train_dummy

,APPLY,CompleteIDCertification,EndLoanApply,Error,Error2,GetCreditInfo,Login,OpenApp,SignUp,StartLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,ViewLoanApplyIntro
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636179,0,0,0,0,0,0,1,0,0,0,0,0,0,0
13636180,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13636181,0,1,0,0,0,0,0,0,0,0,0,0,0,0
13636182,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [131]:
concat_train_dummy = concat_train_fin.join(train_dummy.add_suffix('_1'))

In [132]:
pd.options.display.max_columns = None

In [133]:
concat_train_dummy.drop(['user_id','timestamp','event','mp_os','Session','date','how_long','visit_cumsum','from_last_visit',
                         'before_7days','before_30days','visit_rate','session_sum','session_avg'],axis=1,inplace=True)

In [134]:
train_dummy2 = concat_train_dummy.groupby('application_id').cumsum()

In [135]:
train_dummy2

,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,1,0,1,0,0,0,0,0,0
9,0,0,0,0,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636179,0,0,0,0,3,0,2,1,0,0,0,2,0,0
13636180,0,0,0,0,3,0,2,1,0,0,0,2,0,1
13636181,0,1,0,0,3,0,2,1,0,0,0,2,0,1
13636182,1,1,0,0,3,0,2,1,0,0,0,2,0,1


In [136]:
train_dummy2['event_count_1']=train_dummy2.sum(axis=1)

In [137]:
concat_cumsum_train = pd.concat([concat_train_fin,train_dummy2],axis=1)

In [138]:
concat_cumsum_train

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1
3,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2
7,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
8,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2
9,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636179,1856758.0,879696,2022-03-14 05:37:39,Login,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,2,1,0,0,0,2,0,0,8
13636180,1856758.0,879696,2022-03-14 05:37:55,ViewLoanApplyIntro,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,2,1,0,0,0,2,0,1,9
13636181,1856758.0,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,1,0,0,3,0,2,1,0,0,0,2,0,1,10
13636182,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,1,1,0,0,3,0,2,1,0,0,0,2,0,1,11


#### events변수 만들기(1)(event unique)

In [139]:
event_series = concat_train_fin.groupby('application_id').agg({'event':'unique'}).reset_index()

In [140]:
event_series_new = event_series.copy()

In [141]:
event_series_new

,application_id,event
0,1.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
1,2.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
2,6.0,"[ViewLoanApplyIntro, CompleteIDCertification, ..."
3,14.0,"[Error2, APPLY]"
4,17.0,"[OpenApp, ViewLoanApplyIntro, CompleteIDCertif..."
...,...,...
728855,2167811.0,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
728856,2167817.0,"[OpenApp, Login, UseLoanManage, Error2, StartL..."
728857,2167819.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
728858,2167820.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."


In [142]:
event_series_new.rename(columns={'event':'events_1'},inplace=True)

In [143]:
event_series_new

,application_id,events_1
0,1.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
1,2.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
2,6.0,"[ViewLoanApplyIntro, CompleteIDCertification, ..."
3,14.0,"[Error2, APPLY]"
4,17.0,"[OpenApp, ViewLoanApplyIntro, CompleteIDCertif..."
...,...,...
728855,2167811.0,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
728856,2167817.0,"[OpenApp, Login, UseLoanManage, Error2, StartL..."
728857,2167819.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
728858,2167820.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."


In [144]:
concat_train_fin2 = pd.merge(concat_cumsum_train, event_series_new, on="application_id")

In [145]:
concat_train_fin2.shape

(4297854, 31)

In [146]:
concat_train_fin2

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1
0,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,"[Error2, APPLY]"
1,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]"
2,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
3,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
4,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,3,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4297849,1856758.0,879696,2022-03-14 05:37:39,Login,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,2,1,0,0,0,2,0,0,8,"[Login, UseLoanManage, Error2, OpenApp, ViewLo..."
4297850,1856758.0,879696,2022-03-14 05:37:55,ViewLoanApplyIntro,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,2,1,0,0,0,2,0,1,9,"[Login, UseLoanManage, Error2, OpenApp, ViewLo..."
4297851,1856758.0,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,1,0,0,3,0,2,1,0,0,0,2,0,1,10,"[Login, UseLoanManage, Error2, OpenApp, ViewLo..."
4297852,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,1,1,0,0,3,0,2,1,0,0,0,2,0,1,11,"[Login, UseLoanManage, Error2, OpenApp, ViewLo..."


#### connet_time 변수 만들기 (application_id별 가장 나중 event 시간 - 가장 처음 event 시간)

In [147]:
#해당 user가 APPLY하기까지 걸린 총 시간을 계산한 칼럼 추가
connect_time = pd.DataFrame(concat_train_fin2.groupby('application_id')['timestamp'].max() - concat_train_fin2.groupby('application_id')['timestamp'].min())
connect_time.reset_index(inplace=True)
connect_time.rename(columns={'timestamp':'connect_time'},inplace=True)

In [148]:
concat_train_fin2 = pd.merge(concat_train_fin2, connect_time, on='application_id')
concat_train_fin2.shape

(4297854, 32)

In [149]:
concat_train_fin2['connect_time'].describe()

count                      4297854
mean     0 days 00:05:04.754902563
std      0 days 00:08:39.400431255
min                0 days 00:00:00
25%                0 days 00:01:31
50%                0 days 00:02:25
75%                0 days 00:04:41
max                0 days 12:13:07
Name: connect_time, dtype: object

In [150]:
#log활동 기록이 없어서 event_series에 APPLY만 기록된 경우는 connect_time도 0으로 기록됨.
#concat_train5[concat_train5['connect_time']=='0 days 00:00:00']['events_1'].value_counts()
concat_train_fin2[concat_train_fin2['connect_time']=='0 days 00:00:00']['events_1']

17         [APPLY]
27         [APPLY]
28         [APPLY]
102        [APPLY]
164        [APPLY]
            ...   
4297615    [APPLY]
4297730    [APPLY]
4297788    [APPLY]
4297841    [APPLY]
4297853    [APPLY]
Name: events_1, Length: 88212, dtype: object

#### event 행동 별 횟수 변수 만들기(2)

In [151]:
#같은 application_id 내의 반복적인 행동은 중복이라고 판단하여 제거
df = concat_train_fin2.drop_duplicates(['user_id','application_id','event'],keep="first")

In [152]:
df.shape

(3781682, 32)

In [225]:
df.columns

Index(['application_id', 'user_id', 'timestamp', 'event', 'mp_os', 'Session',
       'date', 'how_long', 'visit_cumsum', 'from_last_visit', 'before_7days',
       'before_30days', 'visit_rate', 'session_sum', 'session_avg', 'APPLY_1',
       'CompleteIDCertification_1', 'EndLoanApply_1', 'Error_1', 'Error2_1',
       'GetCreditInfo_1', 'Login_1', 'OpenApp_1', 'SignUp_1',
       'StartLoanApply_1', 'UseDSRCalc_1', 'UseLoanManage_1',
       'UsePrepayCalc_1', 'ViewLoanApplyIntro_1', 'event_count_1', 'events_1',
       'connect_time'],
      dtype='object')

In [153]:
#처음에 확인한 것과 마찬가지로 train데이터 내에는 application이 총 728,860개 존재
df['application_id'].nunique()

728860

In [154]:
train_dummy3 = pd.get_dummies(df['event'],dummy_na=False)

In [559]:
#train_dummy3.drop(['APPLY'],axis=1,inplace=True)

In [155]:
df2 = df.join(train_dummy3.add_suffix('_2'))

In [156]:
df2.columns

Index(['application_id', 'user_id', 'timestamp', 'event', 'mp_os', 'Session',
       'date', 'how_long', 'visit_cumsum', 'from_last_visit', 'before_7days',
       'before_30days', 'visit_rate', 'session_sum', 'session_avg', 'APPLY_1',
       'CompleteIDCertification_1', 'EndLoanApply_1', 'Error_1', 'Error2_1',
       'GetCreditInfo_1', 'Login_1', 'OpenApp_1', 'SignUp_1',
       'StartLoanApply_1', 'UseDSRCalc_1', 'UseLoanManage_1',
       'UsePrepayCalc_1', 'ViewLoanApplyIntro_1', 'event_count_1', 'events_1',
       'connect_time', 'APPLY_2', 'CompleteIDCertification_2',
       'EndLoanApply_2', 'Error_2', 'Error2_2', 'GetCreditInfo_2', 'Login_2',
       'OpenApp_2', 'SignUp_2', 'StartLoanApply_2', 'UseDSRCalc_2',
       'UseLoanManage_2', 'UsePrepayCalc_2', 'ViewLoanApplyIntro_2'],
      dtype='object')

In [157]:
df2['event'].unique()

array(['Error2', 'APPLY', 'OpenApp', 'GetCreditInfo', 'UseLoanManage',
       'UsePrepayCalc', 'StartLoanApply', 'ViewLoanApplyIntro',
       'CompleteIDCertification', 'Login', 'SignUp', 'EndLoanApply',
       'UseDSRCalc', 'Error'], dtype=object)

In [158]:
train_dummy4 = df2[['application_id','APPLY_2','CompleteIDCertification_2','EndLoanApply_2', 'GetCreditInfo_2', 'Login_2', 
                    'OpenApp_2', 'SignUp_2', 'StartLoanApply_2','UseDSRCalc_2', 'UseLoanManage_2', 'UsePrepayCalc_2', 
                    'ViewLoanApplyIntro_2']].groupby('application_id').cumsum()

In [159]:
train_dummy4

,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4297848,0,0,0,0,1,1,0,0,0,1,0,0
4297850,0,0,0,0,1,1,0,0,0,1,0,1
4297851,0,1,0,0,1,1,0,0,0,1,0,1
4297852,1,1,0,0,1,1,0,0,0,1,0,1


In [160]:
train_dummy4['event_count_2']=train_dummy4.sum(axis=1)

In [161]:
df3 = pd.concat([df, train_dummy4],axis=1)

In [162]:
df3

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2
0,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,"[Error2, APPLY]",0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,0,0
1,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]",0 days 00:03:11,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,0,0,0,0,0,1,0,0,0,0,0,0,1
3,1180616.0,11,2022-03-24 10:54:07,GetCreditInfo,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,0,0,0,1,0,1,0,0,0,0,0,0,2
4,1180616.0,11,2022-03-24 10:54:08,UseLoanManage,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,3,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,0,0,0,1,0,1,0,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4297848,1856758.0,879696,2022-03-14 05:37:32,OpenApp,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,1,1,0,0,0,2,0,0,7,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,0,0,0,0,1,1,0,0,0,1,0,0,3
4297850,1856758.0,879696,2022-03-14 05:37:55,ViewLoanApplyIntro,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,3,0,2,1,0,0,0,2,0,1,9,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,0,0,0,0,1,1,0,0,0,1,0,1,4
4297851,1856758.0,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,0,1,0,0,3,0,2,1,0,0,0,2,0,1,10,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,0,1,0,0,1,1,0,0,0,1,0,1,5
4297852,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.0,0.0,0.0,1,1,0,0,3,0,2,1,0,0,0,2,0,1,11,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,1,1,0,0,1,1,0,0,0,1,0,1,6


In [163]:
df3.head(3)

,application_id,user_id,timestamp,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2
0,248576.0,9,2022-05-21 23:37:58,Error2,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,"[Error2, APPLY]",0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,0,0
1,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]",0 days 00:03:11,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1180616.0,11,2022-03-24 10:53:59,OpenApp,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,0,0,0,0,0,1,0,0,0,0,0,0,1


In [164]:
df3['event'].value_counts()

APPLY                      728860
CompleteIDCertification    596729
ViewLoanApplyIntro         595104
StartLoanApply             512941
OpenApp                    488046
Login                      419264
Error2                     202786
UseLoanManage              131876
EndLoanApply                62474
GetCreditInfo               37629
SignUp                       4666
UsePrepayCalc                 712
UseDSRCalc                    569
Error                          26
Name: event, dtype: int64

#### events변수 만들기(2)

In [165]:
error = df3[df3['event'] == 'Error'].index
df3.drop(error,axis='index',inplace=True)
error2 = df3[df3['event'] == 'Error2'].index
df3.drop(error2,axis='index',inplace=True)

In [166]:
event_series2 = df3.groupby('application_id').agg({'event':'unique'}).reset_index()

In [167]:
event_series2.rename(columns={'event':'events_2'},inplace=True)

In [168]:
event_series2

,application_id,events_2
0,1.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
1,2.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
2,6.0,"[ViewLoanApplyIntro, CompleteIDCertification, ..."
3,14.0,[APPLY]
4,17.0,"[OpenApp, ViewLoanApplyIntro, CompleteIDCertif..."
...,...,...
728855,2167811.0,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
728856,2167817.0,"[OpenApp, Login, UseLoanManage, StartLoanApply..."
728857,2167819.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
728858,2167820.0,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."


In [169]:
train_fin3 = pd.merge(df3, event_series2, on="application_id")

In [226]:
train_fin3.shape

(3578870, 46)

In [170]:
train_fin3.columns

Index(['application_id', 'user_id', 'timestamp', 'event', 'mp_os', 'Session',
       'date', 'how_long', 'visit_cumsum', 'from_last_visit', 'before_7days',
       'before_30days', 'visit_rate', 'session_sum', 'session_avg', 'APPLY_1',
       'CompleteIDCertification_1', 'EndLoanApply_1', 'Error_1', 'Error2_1',
       'GetCreditInfo_1', 'Login_1', 'OpenApp_1', 'SignUp_1',
       'StartLoanApply_1', 'UseDSRCalc_1', 'UseLoanManage_1',
       'UsePrepayCalc_1', 'ViewLoanApplyIntro_1', 'event_count_1', 'events_1',
       'connect_time', 'APPLY_2', 'CompleteIDCertification_2',
       'EndLoanApply_2', 'GetCreditInfo_2', 'Login_2', 'OpenApp_2', 'SignUp_2',
       'StartLoanApply_2', 'UseDSRCalc_2', 'UseLoanManage_2',
       'UsePrepayCalc_2', 'ViewLoanApplyIntro_2', 'event_count_2', 'events_2'],
      dtype='object')

## application_id 당 하나의 행만 남기고 제거

In [171]:
#application_id를 APPLY한 시각만 남기고(기준) 나머지 중복 행 제거
train_fin4 = train_fin3[train_fin3['event']=='APPLY']

In [172]:
train_fin4.shape

(728860, 46)

In [173]:
train_fin4.rename(columns={'timestamp':'insert_time'},inplace=True) #train데이터와 이름 일치시키기

/tmp/ipykernel_32409/3209593557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fin4.rename(columns={'timestamp':'insert_time'},inplace=True) #train데이터와 이름 일치시키기


In [224]:
train_fin4.columns

Index(['application_id', 'user_id', 'insert_time', 'event', 'mp_os', 'Session',
       'date', 'how_long', 'visit_cumsum', 'from_last_visit', 'before_7days',
       'before_30days', 'visit_rate', 'session_sum', 'session_avg', 'APPLY_1',
       'CompleteIDCertification_1', 'EndLoanApply_1', 'Error_1', 'Error2_1',
       'GetCreditInfo_1', 'Login_1', 'OpenApp_1', 'SignUp_1',
       'StartLoanApply_1', 'UseDSRCalc_1', 'UseLoanManage_1',
       'UsePrepayCalc_1', 'ViewLoanApplyIntro_1', 'event_count_1', 'events_1',
       'connect_time', 'APPLY_2', 'CompleteIDCertification_2',
       'EndLoanApply_2', 'GetCreditInfo_2', 'Login_2', 'OpenApp_2', 'SignUp_2',
       'StartLoanApply_2', 'UseDSRCalc_2', 'UseLoanManage_2',
       'UsePrepayCalc_2', 'ViewLoanApplyIntro_2', 'event_count_2', 'events_2'],
      dtype='object')

In [174]:
train_fin4.head(3)

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
0,248576.0,9,2022-05-21 23:41:09,APPLY,android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]",0 days 00:03:11,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
8,1180616.0,11,2022-03-24 10:58:22,APPLY,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,2,0,1,0,1,0,1,1,1,9,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,1,1,0,1,0,1,0,1,0,1,1,1,8,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
14,1751522.0,11,2022-04-20 10:11:03,APPLY,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo...",0 days 00:06:53,1,1,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo..."


#### mp_os 결측치 채우기

In [175]:
train_fin4['mp_os'].value_counts()

Android    489204
iOS        154514
android     60781
iPadOS         77
unknown        11
Name: mp_os, dtype: int64

In [176]:
train_fin4[train_fin4['mp_os'].isna()]['user_id']

15             14
192            64
241            69
260            75
672           218
            ...  
3578144    879547
3578456    879598
3578535    879604
3578771    879685
3578869    879697
Name: user_id, Length: 24273, dtype: int64

#### log데이터 내에 있는 user들은 그 mp_os 정보로 결측치를 채움

In [177]:
train_fin4[train_fin4['mp_os']=='unknown']

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
49444,760375.0,13052,2022-03-18 13:03:18,APPLY,unknown,1.0,2022-03-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:02:07,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
672669,227293.0,165328,2022-05-24 17:26:08,APPLY,unknown,1.0,2022-05-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:02:27,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
987888,536924.0,243710,2022-04-18 12:49:48,APPLY,unknown,1.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,3,"[Error, APPLY]",0 days 00:01:46,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
1400215,908767.0,346408,2022-05-24 15:37:38,APPLY,unknown,1.0,2022-05-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:07:08,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
1619629,1570549.0,400047,2022-04-18 11:20:31,APPLY,unknown,1.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:02:10,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
2303380,1700766.0,567073,2022-04-23 09:48:07,APPLY,unknown,1.0,2022-04-23,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:02:04,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
2303381,1214882.0,567073,2022-04-23 09:52:51,APPLY,unknown,1.0,2022-04-23,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
2722405,1391783.0,669844,2022-04-19 16:53:01,APPLY,unknown,0.0,2022-04-19,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
2722406,2003104.0,669844,2022-05-20 08:49:23,APPLY,unknown,1.0,2022-05-20,32,1.0,31.0,0,0,0.031250,1.0,1.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,"[Error, APPLY]",0 days 00:06:29,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
2722407,1166829.0,669844,2022-05-23 12:50:05,APPLY,unknown,0.0,2022-05-23,35,2.0,3.0,1,1,0.057143,2.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]


In [178]:
log_train2 = pd.read_csv('data/log_train.csv')

In [1005]:
log_train2[log_train2['user_id']==669844]

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
3370163,669844,Login,2022-05-23 16:00:40,Android,3.13.0,2022-05-23
3370164,669844,OpenApp,2022-05-23 16:00:57,Android,3.13.0,2022-05-23
3370165,669844,SignUp,2022-05-23 16:01:56,Android,3.13.0,2022-05-23
3370166,669844,StartLoanApply,2022-05-23 16:02:00,Android,3.13.0,2022-05-23
3370167,669844,ViewLoanApplyIntro,2022-05-23 16:02:00,Android,3.13.0,2022-05-23
3370168,669844,CompleteIDCertification,2022-05-23 16:02:17,Android,3.13.0,2022-05-23
3370169,669844,EndLoanApply,2022-05-23 16:05:31,Android,3.13.0,2022-05-23
9659670,669844,GetCreditInfo,2022-05-20 08:42:54,NaN,NaN,2022-05-20
10470733,669844,StartLoanApply,2022-05-24 10:51:08,Android,3.13.0,2022-05-24
10470735,669844,ViewLoanApplyIntro,2022-05-24 10:51:09,Android,3.13.0,2022-05-24


In [179]:
train_fin4.loc[train_fin4['application_id'] == 760375, 'mp_os'] = 'iOS'
train_fin4.loc[train_fin4['application_id'] == 536924, 'mp_os'] = 'Android'
train_fin4.loc[train_fin4['application_id'] == 1570549, 'mp_os'] = 'Android'
train_fin4.loc[train_fin4['application_id'] == 1391783, 'mp_os'] = 'Android'
train_fin4.loc[train_fin4['application_id'] == 2003104, 'mp_os'] = 'Android'
train_fin4.loc[train_fin4['application_id'] == 1166829, 'mp_os'] = 'Android'
train_fin4.loc[train_fin4['application_id'] == 1808975, 'mp_os'] = 'Android'

In [180]:
train_fin4['mp_os'].value_counts(dropna=False)

Android    489210
iOS        154515
android     60781
NaN         24273
iPadOS         77
unknown         4
Name: mp_os, dtype: int64

In [181]:
train_fin4[train_fin4['mp_os'].isna()]['user_id'].values

array([    14,     64,     69, ..., 879604, 879685, 879697])

In [182]:
no_mp = train_fin4[train_fin4['mp_os'].isna()]['user_id'].values
no_mp = set(list(no_mp))

In [183]:
log_train2[log_train2['user_id']==879697]

,user_id,event,timestamp,mp_os,mp_app_version,date_cd


In [184]:
mp_user = pd.DataFrame(log_train2[log_train2['user_id'].isin(no_mp)])

In [185]:
mp_user['user_id'].nunique()

0

In [186]:
mp_user['mp_os'].value_counts() 

Series([], Name: mp_os, dtype: int64)

In [187]:
train_fin4.loc[train_fin4['mp_os'] == 'android', 'mp_os'] = 'Android' #android로 표기되어 있던 것 Android로 일치시킴
train_fin4.loc[train_fin4['mp_os'] == 'iPadOS', 'mp_os'] = 'iOS' #iPadOS로 표기되어 있던 것 iOS로 변경

In [188]:
train_fin4['mp_os'].value_counts(dropna=False)

Android    549991
iOS        154592
NaN         24273
unknown         4
Name: mp_os, dtype: int64

In [189]:
train_fin4

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
0,248576.0,9,2022-05-21 23:41:09,APPLY,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]",0 days 00:03:11,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
8,1180616.0,11,2022-03-24 10:58:22,APPLY,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,2,0,1,0,1,0,1,1,1,9,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,1,1,0,1,0,1,0,1,0,1,1,1,8,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
14,1751522.0,11,2022-04-20 10:11:03,APPLY,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo...",0 days 00:06:53,1,1,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo..."
15,1508745.0,14,2022-04-18 14:37:09,APPLY,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
19,1990215.0,17,2022-03-06 10:46:59,APPLY,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI...",0 days 00:02:30,1,1,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578857,716861.0,879693,2022-05-20 16:33:03,APPLY,iOS,1.0,2022-05-20,8,2.0,3.0,1,1,0.250000,4.0,2.0,1,1,0,0,0,0,0,1,0,2,0,0,0,2,7,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",0 days 00:01:48,1,1,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
3578861,1774704.0,879695,2022-05-27 12:50:55,APPLY,iOS,1.0,2022-05-27,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI...",0 days 00:02:23,1,1,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
3578862,2149910.0,879696,2022-03-14 05:23:30,APPLY,Android,0.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
3578868,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,3,0,2,1,0,0,0,2,0,1,11,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,1,1,0,0,1,1,0,0,0,1,0,1,6,"[Login, UseLoanManage, OpenApp, ViewLoanApplyI..."


#### 누적 event 행동 횟수 구하기

In [190]:
train_fin_real = train_fin4.copy()

In [191]:
train_fin_real

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,APPLY_2,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
0,248576.0,9,2022-05-21 23:41:09,APPLY,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,"[Error2, APPLY]",0 days 00:03:11,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
8,1180616.0,11,2022-03-24 10:58:22,APPLY,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,2,0,1,0,1,0,1,1,1,9,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre...",0 days 00:04:23,1,1,0,1,0,1,0,1,0,1,1,1,8,"[OpenApp, GetCreditInfo, UseLoanManage, UsePre..."
14,1751522.0,11,2022-04-20 10:11:03,APPLY,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo...",0 days 00:06:53,1,1,0,1,0,1,0,0,0,1,0,1,6,"[OpenApp, GetCreditInfo, UseLoanManage, ViewLo..."
15,1508745.0,14,2022-04-18 14:37:09,APPLY,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
19,1990215.0,17,2022-03-06 10:46:59,APPLY,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI...",0 days 00:02:30,1,1,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578857,716861.0,879693,2022-05-20 16:33:03,APPLY,iOS,1.0,2022-05-20,8,2.0,3.0,1,1,0.250000,4.0,2.0,1,1,0,0,0,0,0,1,0,2,0,0,0,2,7,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",0 days 00:01:48,1,1,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
3578861,1774704.0,879695,2022-05-27 12:50:55,APPLY,iOS,1.0,2022-05-27,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI...",0 days 00:02:23,1,1,0,0,0,0,0,1,0,0,0,1,4,"[StartLoanApply, ViewLoanApplyIntro, CompleteI..."
3578862,2149910.0,879696,2022-03-14 05:23:30,APPLY,Android,0.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY],0 days 00:00:00,1,0,0,0,0,0,0,0,0,0,0,0,1,[APPLY]
3578868,1856758.0,879696,2022-03-14 05:40:38,APPLY,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,1,0,0,3,0,2,1,0,0,0,2,0,1,11,"[Login, UseLoanManage, Error2, OpenApp, ViewLo...",0 days 00:05:04,1,1,0,0,1,1,0,0,0,1,0,1,6,"[Login, UseLoanManage, OpenApp, ViewLoanApplyI..."


In [192]:
train_fin_real['APPLY_cumsum_1'] = train_fin_real.groupby('user_id')['APPLY_1'].cumsum()
train_fin_real['CompleteIDCertification_cumsum_1'] = train_fin_real.groupby('user_id')['CompleteIDCertification_1'].cumsum()
train_fin_real['EndLoanApply_cumsum_1'] = train_fin_real.groupby('user_id')['EndLoanApply_1'].cumsum()
train_fin_real['GetCreditInfo_cumsum_1'] = train_fin_real.groupby('user_id')['GetCreditInfo_1'].cumsum()
train_fin_real['Login_cumsum_1'] = train_fin_real.groupby('user_id')['Login_1'].cumsum()
train_fin_real['OpenApp_cumsum_1'] = train_fin_real.groupby('user_id')['OpenApp_1'].cumsum()
train_fin_real['SignUp_cumsum_1'] = train_fin_real.groupby('user_id')['SignUp_1'].cumsum()
train_fin_real['StartLoanApply_cumsum_1'] = train_fin_real.groupby('user_id')['StartLoanApply_1'].cumsum()
train_fin_real['UseDSRCalc_cumsum_1'] = train_fin_real.groupby('user_id')['UseDSRCalc_1'].cumsum()
train_fin_real['UseLoanManage_cumsum_1'] = train_fin_real.groupby('user_id')['UseLoanManage_1'].cumsum()
train_fin_real['UsePrepayCalc_cumsum_1'] = train_fin_real.groupby('user_id')['UsePrepayCalc_1'].cumsum()
train_fin_real['ViewLoanApplyIntro_cumsum_1'] = train_fin_real.groupby('user_id')['ViewLoanApplyIntro_1'].cumsum()

In [193]:
train_fin_real['APPLY_cumsum_2'] = train_fin_real.groupby('user_id')['APPLY_2'].cumsum()
train_fin_real['CompleteIDCertification_cumsum_2'] = train_fin_real.groupby('user_id')['CompleteIDCertification_2'].cumsum()
train_fin_real['EndLoanApply_cumsum_2'] = train_fin_real.groupby('user_id')['EndLoanApply_2'].cumsum()
train_fin_real['GetCreditInfo_cumsum_2'] = train_fin_real.groupby('user_id')['GetCreditInfo_2'].cumsum()
train_fin_real['Login_cumsum_2'] = train_fin_real.groupby('user_id')['Login_2'].cumsum()
train_fin_real['OpenApp_cumsum_2'] = train_fin_real.groupby('user_id')['OpenApp_2'].cumsum()
train_fin_real['SignUp_cumsum_2'] = train_fin_real.groupby('user_id')['SignUp_2'].cumsum()
train_fin_real['StartLoanApply_cumsum_2'] = train_fin_real.groupby('user_id')['StartLoanApply_2'].cumsum()
train_fin_real['UseDSRCalc_cumsum_2'] = train_fin_real.groupby('user_id')['UseDSRCalc_2'].cumsum()
train_fin_real['UseLoanManage_cumsum_2'] = train_fin_real.groupby('user_id')['UseLoanManage_2'].cumsum()
train_fin_real['UsePrepayCalc_cumsum_2'] = train_fin_real.groupby('user_id')['UsePrepayCalc_2'].cumsum()
train_fin_real['ViewLoanApplyIntro_cumsum_2'] = train_fin_real.groupby('user_id')['ViewLoanApplyIntro_2'].cumsum()

In [194]:
train_fin_real.drop(['APPLY_1','APPLY_2','APPLY_cumsum_2'],axis=1,inplace=True)

In [195]:
train_fin_real['APPLY_cumsum_1'] = train_fin_real['APPLY_cumsum_1']-1

In [196]:
train_fin_real.rename(columns={'APPLY_cumsum_1':'before_apply_count'},inplace=True)

In [197]:
pd.set_option('display.max_columns', None) 
train_fin_real[train_fin_real['user_id']==377301].sort_values('insert_time')

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
1525920,1817572.0,377301,2022-03-18 15:48:26,APPLY,Android,6.0,2022-03-18,18,16.0,1.0,1,1,0.888889,71.0,4.437500,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:34,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0,1,2,0,1,1,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1
1525927,1746550.0,377301,2022-04-06 14:30:38,APPLY,Android,3.0,2022-04-06,37,35.0,1.0,1,1,0.945946,168.0,4.800000,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:38,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",1,2,4,0,2,2,0,2,0,0,0,2,2,2,0,2,2,0,2,0,0,0,2
1525934,1622040.0,377301,2022-04-06 21:47:02,APPLY,Android,4.0,2022-04-06,37,35.0,1.0,1,1,0.945946,168.0,4.800000,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:41,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",2,3,6,0,3,3,0,3,0,0,0,3,3,3,0,3,3,0,3,0,0,0,3
1525941,1947380.0,377301,2022-04-07 13:15:36,APPLY,Android,3.0,2022-04-07,38,36.0,1.0,1,1,0.947368,172.0,4.777778,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:33,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",3,4,8,0,4,4,0,4,0,0,0,4,4,4,0,4,4,0,4,0,0,0,4
1525948,1541472.0,377301,2022-04-07 16:51:42,APPLY,Android,4.0,2022-04-07,38,36.0,1.0,1,1,0.947368,172.0,4.777778,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:35,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",4,5,9,0,5,5,0,5,0,0,0,5,5,5,0,5,5,0,5,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527642,1752261.0,377301,2022-05-31 08:54:00,APPLY,Android,1.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,0,0,0,0,1,1,0,1,0,0,0,1,6,"[OpenApp, Login, StartLoanApply, ViewLoanApply...",0 days 00:00:43,1,0,0,1,1,0,1,0,0,0,1,6,"[OpenApp, Login, StartLoanApply, ViewLoanApply...",262,265,273,0,246,250,0,259,0,4,0,271,262,219,0,237,237,0,245,0,4,0,262
1527649,1884174.0,377301,2022-05-31 10:02:57,APPLY,Android,2.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:01:11,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",263,266,274,0,247,251,0,260,0,4,0,272,263,220,0,238,238,0,246,0,4,0,263
1527656,120423.0,377301,2022-05-31 12:54:06,APPLY,Android,3.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:01:08,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanA

In [198]:
train_fin_real[train_fin_real['user_id']==377301]

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
1525920,1817572.0,377301,2022-03-18 15:48:26,APPLY,Android,6.0,2022-03-18,18,16.0,1.0,1,1,0.888889,71.0,4.437500,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:34,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0,1,2,0,1,1,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1
1525927,1746550.0,377301,2022-04-06 14:30:38,APPLY,Android,3.0,2022-04-06,37,35.0,1.0,1,1,0.945946,168.0,4.800000,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:38,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",1,2,4,0,2,2,0,2,0,0,0,2,2,2,0,2,2,0,2,0,0,0,2
1525934,1622040.0,377301,2022-04-06 21:47:02,APPLY,Android,4.0,2022-04-06,37,35.0,1.0,1,1,0.945946,168.0,4.800000,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:41,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",2,3,6,0,3,3,0,3,0,0,0,3,3,3,0,3,3,0,3,0,0,0,3
1525941,1947380.0,377301,2022-04-07 13:15:36,APPLY,Android,3.0,2022-04-07,38,36.0,1.0,1,1,0.947368,172.0,4.777778,1,2,0,0,0,1,1,0,1,0,0,0,1,8,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:33,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",3,4,8,0,4,4,0,4,0,0,0,4,4,4,0,4,4,0,4,0,0,0,4
1525948,1541472.0,377301,2022-04-07 16:51:42,APPLY,Android,4.0,2022-04-07,38,36.0,1.0,1,1,0.947368,172.0,4.777778,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:00:35,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",4,5,9,0,5,5,0,5,0,0,0,5,5,5,0,5,5,0,5,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527642,1752261.0,377301,2022-05-31 08:54:00,APPLY,Android,1.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,0,0,0,0,1,1,0,1,0,0,0,1,6,"[OpenApp, Login, StartLoanApply, ViewLoanApply...",0 days 00:00:43,1,0,0,1,1,0,1,0,0,0,1,6,"[OpenApp, Login, StartLoanApply, ViewLoanApply...",262,265,273,0,246,250,0,259,0,4,0,271,262,219,0,237,237,0,245,0,4,0,262
1527649,1884174.0,377301,2022-05-31 10:02:57,APPLY,Android,2.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:01:11,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",263,266,274,0,247,251,0,260,0,4,0,272,263,220,0,238,238,0,246,0,4,0,263
1527656,120423.0,377301,2022-05-31 12:54:06,APPLY,Android,3.0,2022-05-31,92,90.0,1.0,1,1,0.978261,482.0,5.355556,1,1,0,0,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanApply,...",0 days 00:01:08,1,1,0,1,1,0,1,0,0,0,1,7,"[OpenApp, Login, StartLoanApply, EndLoanA

In [199]:
train_fin_real.drop(['event'],axis=1,inplace=True)

### events_1, events_2 처리하기

In [200]:
train_fin_real2 = train_fin_real.copy()

In [201]:
train_fin_real2 = train_fin_real2.astype({'events_1':'str','events_2':'str'})

In [202]:
train_fin_real2.head()

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
0,248576.0,9,2022-05-21 23:41:09,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,['Error2' 'APPLY'],0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,1,['APPLY'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1180616.0,11,2022-03-24 10:58:22,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,2,0,1,0,1,0,1,1,1,9,['OpenApp' 'GetCreditInfo' 'UseLoanManage' 'Us...,0 days 00:04:23,1,0,1,0,1,0,1,0,1,1,1,8,['OpenApp' 'GetCreditInfo' 'UseLoanManage' 'Us...,0,1,0,2,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1
14,1751522.0,11,2022-04-20 10:11:03,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,0,0,0,1,0,1,0,0,0,1,0,1,6,['OpenApp' 'GetCreditInfo' 'UseLoanManage' 'Vi...,0 days 00:06:53,1,0,1,0,1,0,0,0,1,0,1,6,['OpenApp' 'GetCreditInfo' 'UseLoanManage' 'Vi...,1,2,0,3,0,2,0,1,0,2,1,2,2,0,2,0,2,0,1,0,2,1,2
15,1508745.0,14,2022-04-18 14:37:09,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,['APPLY'],0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,['APPLY'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,1990215.0,17,2022-03-06 10:46:59,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,['StartLoanApply' 'ViewLoanApplyIntro' 'Comple...,0 days 00:02:30,1,0,0,0,0,0,1,0,0,0,1,4,['StartLoanApply' 'ViewLoanApplyIntro' 'Comple...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1


In [203]:
train_fin_real2["events_1"]= train_fin_real2["events_1"].str.replace(pat=r'[', repl=r'', regex=True)
train_fin_real2["events_1"]= train_fin_real2["events_1"].str.replace(pat=r']', repl=r'', regex=True)
train_fin_real2["events_1"]= train_fin_real2["events_1"].str.replace(pat=r"'", repl=r'', regex=True)
train_fin_real2["events_1"]= train_fin_real2["events_1"].str.replace(pat=r" ", repl=r'_', regex=True)
train_fin_real2["events_1"]= train_fin_real2["events_1"].str.replace(pat=r"\n", repl=r'', regex=True)

In [204]:
train_fin_real2["events_2"]= train_fin_real2["events_2"].str.replace(pat=r'[', repl=r'', regex=True)
train_fin_real2["events_2"]= train_fin_real2["events_2"].str.replace(pat=r']', repl=r'', regex=True)
train_fin_real2["events_2"]= train_fin_real2["events_2"].str.replace(pat=r"'", repl=r'', regex=True)
train_fin_real2["events_2"]= train_fin_real2["events_2"].str.replace(pat=r" ", repl=r'_', regex=True)
train_fin_real2["events_2"]= train_fin_real2["events_2"].str.replace(pat=r"\n", repl=r'', regex=True)

In [205]:
train_fin_real2.head()

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
0,248576.0,9,2022-05-21 23:41:09,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,Error2_APPLY,0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1180616.0,11,2022-03-24 10:58:22,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,2,0,1,0,1,0,1,1,1,9,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0 days 00:04:23,1,0,1,0,1,0,1,0,1,1,1,8,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0,1,0,2,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1
14,1751522.0,11,2022-04-20 10:11:03,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,0,0,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,0 days 00:06:53,1,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,1,2,0,3,0,2,0,1,0,2,1,2,2,0,2,0,2,0,1,0,2,1,2
15,1508745.0,14,2022-04-18 14:37:09,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,1990215.0,17,2022-03-06 10:46:59,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0 days 00:02:30,1,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1


In [206]:
train_fin_real2.shape

(728860, 66)

In [231]:
pd.DataFrame(train_fin_real2['events_1'].value_counts()).head(5)

,events_1
OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,198322
APPLY,88186
OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,65500
Error2_APPLY,42603
OpenApp_Login_Error2_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,34947


In [207]:
pd.DataFrame(train_fin_real2['events_2'].value_counts()).head(5)

,events_2
OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,248956
APPLY,130797
OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,65521
OpenApp_Login_UseLoanManage_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,46885
StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY,32056


## events 변수에 따라 그룹 나누기

In [208]:
def event1(x) :
    if x == 'OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY' :
        return 1
    
    elif x == 'APPLY':
        return 2
    
    elif x == 'OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 3
    
    elif x == 'Error2_APPLY':
        return 4
    
    elif x == 'OpenApp_Login_Error2_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 5
    
    else :
        return 6

train_fin_real2["event_group_1"] = train_fin_real2["events_1"].apply(lambda x : event1(x))

In [209]:
train_fin_real2["event_group_1"].value_counts()

6    299302
1    198322
2     88186
3     65500
4     42603
5     34947
Name: event_group_1, dtype: int64

In [210]:
def event2(x) :
    if x == 'OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY' :
        return 1
    
    elif x == 'APPLY':
        return 2
    
    elif x == 'OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 3
    
    elif x == 'OpenApp_Login_UseLoanManage_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 4
    
    elif x == 'StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 5
    
    else :
        return 6

train_fin_real2["event_group_2"] = train_fin_real2["events_2"].apply(lambda x : event2(x))

In [211]:
train_fin_real2["event_group_2"].value_counts()

1    248956
6    204645
2    130797
3     65521
4     46885
5     32056
Name: event_group_2, dtype: int64

In [212]:
train_fin_real2.shape

(728860, 68)

In [213]:
train_fin_real3 = pd.merge(train_fin_real2, concat_train7, on=['application_id','user_id'])

In [214]:
train_fin_real3

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2,event_group_1,event_group_2,from_last_visit_time
0,248576.0,9,2022-05-21 23:41:09,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,Error2_APPLY,0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,NaT
1,1180616.0,11,2022-03-24 10:58:22,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,2,0,1,0,1,0,1,1,1,9,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0 days 00:04:23,1,0,1,0,1,0,1,0,1,1,1,8,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0,1,0,2,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,6,6,NaT
2,1751522.0,11,2022-04-20 10:11:03,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,0,0,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,0 days 00:06:53,1,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,1,2,0,3,0,2,0,1,0,2,1,2,2,0,2,0,2,0,1,0,2,1,2,6,6,26 days 22:58:39
3,1508745.0,14,2022-04-18 14:37:09,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,NaT
4,1990215.0,17,2022-03-06 10:46:59,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0 days 00:02:30,1,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,6,5,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728855,716861.0,879693,2022-05-20 16:33:03,iOS,1.0,2022-05-20,8,2.0,3.0,1,1,0.250000,4.0,2.0,1,0,0,0,0,0,1,0,2,0,0,0,2,7,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0 days 00:01:48,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,2,3,0,0,0,1,0,4,0,0,0,4,3,0,0,0,1,0,3,0,0,0,3,3,3,3 days 02:09:34
728856,1774704.0,879695,2022-05-27 12:50:55,iOS,1.0,2022-05-27,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0 days 00:02:23,1,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,6,5,NaT
728857,2149910.0,879696,2022-03-14 05:23:30,Android,0.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,NaT
728858,1856758.0,879696,2022-03-14 05:40:38,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,3,0,2,1,0,0,0,2,0,1,11,Login_UseLoanManage_Error2_OpenApp_ViewLoanApp...,0 days 00:05:04,1,0,0,1,1,0,0,0,1,0,1,6,Login_UseLoanManage_OpenApp_ViewLoanApplyIntro...,1,1,0,0,2,1,0,0,0,2,0,1,1,0,0,1,1,0,0,0,1,0,1,6,6,0 days 00:12:04


In [215]:
train_fin_real3['from_last_visit_time'].fillna('0 days 00:00:00',inplace=True)

In [216]:
train_fin_real3.isna().sum()

application_id                     0
user_id                            0
insert_time                        0
mp_os                          24273
Session                            0
                               ...  
UsePrepayCalc_cumsum_2             0
ViewLoanApplyIntro_cumsum_2        0
event_group_1                      0
event_group_2                      0
from_last_visit_time               0
Length: 69, dtype: int64

In [217]:
train_fin_real3

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2,event_group_1,event_group_2,from_last_visit_time
0,248576.0,9,2022-05-21 23:41:09,Android,1.0,2022-05-21,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,Error2_APPLY,0 days 00:03:11,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0 days 00:00:00
1,1180616.0,11,2022-03-24 10:58:22,iOS,1.0,2022-03-24,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,2,0,1,0,1,0,1,1,1,9,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0 days 00:04:23,1,0,1,0,1,0,1,0,1,1,1,8,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,0,1,0,2,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,6,6,0 days 00:00:00
2,1751522.0,11,2022-04-20 10:11:03,iOS,1.0,2022-04-20,28,1.0,27.0,0,1,0.035714,1.0,1.0,1,0,0,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,0 days 00:06:53,1,0,1,0,1,0,0,0,1,0,1,6,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,1,2,0,3,0,2,0,1,0,2,1,2,2,0,2,0,2,0,1,0,2,1,2,6,6,26 days 22:58:39
3,1508745.0,14,2022-04-18 14:37:09,NaN,0.0,2022-04-18,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0 days 00:00:00
4,1990215.0,17,2022-03-06 10:46:59,iOS,1.0,2022-03-06,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0 days 00:02:30,1,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,6,5,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728855,716861.0,879693,2022-05-20 16:33:03,iOS,1.0,2022-05-20,8,2.0,3.0,1,1,0.250000,4.0,2.0,1,0,0,0,0,0,1,0,2,0,0,0,2,7,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0 days 00:01:48,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,2,3,0,0,0,1,0,4,0,0,0,4,3,0,0,0,1,0,3,0,0,0,3,3,3,3 days 02:09:34
728856,1774704.0,879695,2022-05-27 12:50:55,iOS,1.0,2022-05-27,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0 days 00:02:23,1,0,0,0,0,0,1,0,0,0,1,4,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,6,5,0 days 00:00:00
728857,2149910.0,879696,2022-03-14 05:23:30,Android,0.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0 days 00:00:00
728858,1856758.0,879696,2022-03-14 05:40:38,Android,1.0,2022-03-14,1,0.0,0.0,0,0,0.000000,0.0,0.0,1,0,0,3,0,2,1,0,0,0,2,0,1,11,Login_UseLoanManage_Error2_OpenApp_ViewLoanApp...,0 days 00:05:04,1,0,0,1,1,0,0,0,1,0,1,6,Login_UseLoanManage_OpenApp_ViewLoanApplyIntro...,1,

In [218]:
train_fin_real3.shape

(728860, 69)

In [219]:
train_fin_real3.rename(columns={'event_count_1':'event_count1',
                               'event_count_2':'event_count2',
                               'event_group_1':'event_group1',
                               'event_group_2':'event_group2'},inplace=True)

In [220]:
train_fin_real3 = train_fin_real3[['application_id', 'user_id', 'insert_time', 'mp_os', 'Session', 'date', 'how_long', 
                                   'visit_cumsum', 'from_last_visit', 'visit_rate','session_sum', 'session_avg',
                                   'CompleteIDCertification_1','EndLoanApply_1', 'Error_1', 'Error2_1', 'GetCreditInfo_1',
                                   'Login_1','OpenApp_1', 'SignUp_1', 'StartLoanApply_1', 'UseDSRCalc_1','UseLoanManage_1', 
                                   'UsePrepayCalc_1', 'ViewLoanApplyIntro_1','events_1',
                                   'CompleteIDCertification_cumsum_1', 'EndLoanApply_cumsum_1','GetCreditInfo_cumsum_1', 
                                   'Login_cumsum_1', 'OpenApp_cumsum_1','SignUp_cumsum_1', 'StartLoanApply_cumsum_1',
                                   'UseDSRCalc_cumsum_1','UseLoanManage_cumsum_1','UsePrepayCalc_cumsum_1','ViewLoanApplyIntro_cumsum_1',
                                   'CompleteIDCertification_2','EndLoanApply_2', 'GetCreditInfo_2', 'Login_2',
                                   'OpenApp_2', 'SignUp_2', 'StartLoanApply_2', 'UseDSRCalc_2','UseLoanManage_2', 'UsePrepayCalc_2',
                                   'ViewLoanApplyIntro_2','events_2','CompleteIDCertification_cumsum_2','EndLoanApply_cumsum_2',
                                   'GetCreditInfo_cumsum_2', 'Login_cumsum_2','OpenApp_cumsum_2', 'SignUp_cumsum_2',
                                   'StartLoanApply_cumsum_2','UseDSRCalc_cumsum_2', 'UseLoanManage_cumsum_2','UsePrepayCalc_cumsum_2',
                                   'ViewLoanApplyIntro_cumsum_2','connect_time','from_last_visit_time','before_apply_count',
                                  'before_7days','before_30days','event_count1','event_count2','event_group1','event_group2']]

In [221]:
train_fin_real3.shape

(728860, 69)

In [222]:
train_fin_real3

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,events_1,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,events_2,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2,connect_time,from_last_visit_time,before_apply_count,before_7days,before_30days,event_count1,event_count2,event_group1,event_group2
0,248576.0,9,2022-05-21 23:41:09,Android,1.0,2022-05-21,1,0.0,0.0,0.000000,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,Error2_APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPLY,0,0,0,0,0,0,0,0,0,0,0,0 days 00:03:11,0 days 00:00:00,0,0,0,2,1,4,2
1,1180616.0,11,2022-03-24 10:58:22,iOS,1.0,2022-03-24,1,0.0,0.0,0.000000,0.0,0.0,1,0,0,0,2,0,1,0,1,0,1,1,1,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,1,0,2,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,OpenApp_GetCreditInfo_UseLoanManage_UsePrepayC...,1,0,1,0,1,0,1,0,1,1,1,0 days 00:04:23,0 days 00:00:00,0,0,0,9,8,6,6
2,1751522.0,11,2022-04-20 10:11:03,iOS,1.0,2022-04-20,28,1.0,27.0,0.035714,1.0,1.0,1,0,0,0,1,0,1,0,0,0,1,0,1,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,2,0,3,0,2,0,1,0,2,1,2,1,0,1,0,1,0,0,0,1,0,1,OpenApp_GetCreditInfo_UseLoanManage_ViewLoanAp...,2,0,2,0,2,0,1,0,2,1,2,0 days 00:06:53,26 days 22:58:39,1,0,1,6,6,6,6
3,1508745.0,14,2022-04-18 14:37:09,NaN,0.0,2022-04-18,1,0.0,0.0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPLY,0,0,0,0,0,0,0,0,0,0,0,0 days 00:00:00,0 days 00:00:00,0,0,0,1,1,2,2
4,1990215.0,17,2022-03-06 10:46:59,iOS,1.0,2022-03-06,1,0.0,0.0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,1,0,0,0,0,0,1,0,0,0,1,0 days 00:02:30,0 days 00:00:00,0,0,0,4,4,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728855,716861.0,879693,2022-05-20 16:33:03,iOS,1.0,2022-05-20,8,2.0,3.0,0.250000,4.0,2.0,1,0,0,0,0,0,1,0,2,0,0,0,2,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,3,0,0,0,1,0,4,0,0,0,4,1,0,0,0,1,0,1,0,0,0,1,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,3,0,0,0,1,0,3,0,0,0,3,0 days 00:01:48,3 days 02:09:34,2,1,1,7,5,3,3
728856,1774704.0,879695,2022-05-27 12:50:55,iOS,1.0,2022-05-27,1,0.0,0.0,0.000000,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,StartLoanApply_ViewLoanApplyIntro_CompleteIDCe...,1,0,0,0,0,0,1,0,0,0,1,0 days 00:02:23,0 days 00:00:00,0,0,0,4,4,6,5
728857,2149910.0,879696,2022-03-14 05:23:30,Android,0.0,2022-03-14,1,0.0,0.0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPLY,0,0,0,0,0,0,0,0,0,0,0,0 days 00:00:00,0 days 00:00:00,0,0,0,1,1,2,2
728858,1856758.0,879696,2022-03-14 05:40:38,Android,1.0,2022-03-14,1,0.0,0.0,0.000000,0.0,0.0,1,0,0,3,0,2,1,0,0,0,2,0,1,Login_UseLoanManage_Error2_OpenApp_ViewLoanApp...,1,0,0,2,1,0,0,0,2,0,1,1,0,0,1,1,0,0,0,1,0,1,Login_UseLoanManage_OpenApp_ViewLoanApplyIntro...,1,0,0,1,1

In [223]:
train_fin_real3.to_csv("log_train_fin(realreal).csv",index=False)

## 확인